# AKS 클러스터 실습 가이드
이 노트북에서는 Azure Kubernetes Service(AKS) 클러스터 생성, 쿠버네티스 및 AKS의 기본 개념, 컨테이너 이미지 배포, 실행, 확장, 모니터링 실습을 진행합니다.

## 1. 쿠버네티스와 AKS 기본 개념 및 이점

### 쿠버네티스란?
쿠버네티스(Kubernetes)는 컨테이너화된 애플리케이션의 배포, 확장, 관리를 자동화하는 오픈소스 플랫폼입니다.

#### 쿠버네티스의 주요 이점
- **자동화된 배포 및 확장**
- **자가 치유(Self-healing)**
- **서비스 디스커버리 및 로드밸런싱**
- **롤링 업데이트 및 롤백**

### AKS란?
AKS(Azure Kubernetes Service)는 Azure에서 제공하는 관리형 쿠버네티스 서비스로, 클러스터 관리의 복잡성을 줄여줍니다.

#### AKS의 주요 이점
- **간편한 클러스터 관리**
- **Azure와의 통합**
- **자동 업그레이드 및 패치**

## 2. AKS 클러스터 생성 실습
Azure CLI를 사용하여 AKS 클러스터를 생성합니다.

### 관리형 Prometheus 및 Grafana 활성화
AKS 클러스터 생성 시 Azure Monitor의 관리형 Prometheus와 Grafana를 함께 활성화할 수 있습니다.
아래 예시처럼 `--enable-addons` 옵션에 `monitoring`과 `azure-monitor-metrics`를 추가합니다.

In [ ]:
# 리소스 그룹 생성
az group create --name aks-lab-rg --location koreacentral

In [ ]:
# 관리형 Prometheus 및 Grafana가 활성화된 AKS 클러스터 생성 예시
az aks create \
  --resource-group aks-lab-rg \
  --name aks-lab-cluster \
  --node-count 2 \
  --enable-addons monitoring,azure-monitor-metrics \
  --generate-ssh-keys

In [ ]:
# 클러스터 인증 정보 가져오기
az aks get-credentials --resource-group aks-lab-rg --name aks-lab-cluster

## 3. 컨테이너 이미지 AKS에 배포하기
앞서 ACR에 푸시한 이미지를 AKS에 배포합니다.

In [ ]:
# deployment.yaml 예시
apiVersion: apps/v1
kind: Deployment
metadata:
  name: app
spec:
  replicas: 2
  selector:
    matchLabels:
      app: app
  template:
    metadata:
      labels:
        app: app
    spec:
      containers:
      - name: app
        image: myacr.azurecr.io/myapp:latest
        ports:
        - containerPort: 8080

In [ ]:
# Deployment 생성
kubectl apply -f deployment.yaml

# Service 생성 (LoadBalancer)
kubectl expose deployment app --type=LoadBalancer --port=80 --target-port=5000

In [ ]:
# 외부 IP 확인
kubectl get service app

## 4. 애플리케이션 확장
replica 수를 조정하여 애플리케이션을 확장할 수 있습니다.

In [ ]:
# replica 수를 4로 확장
kubectl scale deployment app --replicas=4

### HPA(Horizontal Pod Autoscaler)로 오토스케일링 적용하기
쿠버네티스 HPA(Horizontal Pod Autoscaler)를 활용하면 CPU 사용률 등 메트릭에 따라 파드 수를 자동으로 조정할 수 있습니다.

1. **metrics-server 설치** (AKS는 기본 제공):
   - AKS에서는 metrics-server가 기본적으로 설치되어 있으므로 별도 설치가 필요하지 않습니다.

2. **HPA 생성 명령어 예시**
```bash
kubectl autoscale deployment app --cpu-percent=50 --min=2 --max=10
```

- 위 명령은 CPU 사용률이 50%를 초과하면 파드 수를 2~10개 사이에서 자동으로 조정합니다.

3. **HPA 상태 확인**
```bash
kubectl get hpa
```

### HPA 오토스케일링 테스트 방법
HPA가 정상적으로 동작하는지 확인하려면, 파드에 부하를 인위적으로 발생시켜 오토스케일링이 동작하는지 관찰할 수 있습니다.

1. **stress 도구 설치** (app 파드에 설치):
```bash
kubectl exec -it <app-pod-name> -- apt-get update
kubectl exec -it <app-pod-name> -- apt-get install -y stress
```

2. **CPU 부하 발생**:
```bash
kubectl exec -it <app-pod-name> -- stress --cpu 1 --timeout 300
```

3. **HPA 상태 모니터링**:
```bash
kubectl get hpa -w
```

- 위 과정을 통해 CPU 사용률이 증가하면 HPA가 자동으로 파드 수를 늘리는 것을 확인할 수 있습니다.

 > 참고: `<app-pod-name>`은 실제 app 파드 이름으로 대체하세요. 파드 이름은 `kubectl get pods` 명령으로 확인할 수 있습니다.

## 5. 모니터링
AKS는 Azure Monitor와 통합되어 클러스터 및 애플리케이션 상태를 모니터링할 수 있습니다.

Azure Portal에서 AKS 리소스를 선택하고, 모니터링 탭에서 로그와 메트릭을 확인하세요.

### 관리형 Grafana 대시보드에서 Prometheus 표준 대시보드 조회하기
AKS에서 관리형 Prometheus와 Grafana를 활성화했다면, Azure Portal에서 관리형 Grafana 인스턴스에 접근할 수 있습니다.

1. **Azure Portal 접속**: [https://portal.azure.com](https://portal.azure.com) 에 로그인합니다.
2. **Grafana 리소스 검색**: 상단 검색창에 `Grafana`를 입력하고, 생성된 Managed Grafana 리소스를 선택합니다.
3. **Grafana 대시보드 열기**: 'Grafana 작업 영역'에서 'Grafana 대시보드' 버튼을 클릭하여 대시보드에 접속합니다.
4. **Prometheus 데이터 소스 확인**: 좌측 메뉴에서 'Connections > Data sources'로 이동하여 Prometheus가 연결되어 있는지 확인합니다.
5. **표준 대시보드 조회**: 좌측 메뉴에서 'Dashboards'를 클릭하고, 'Browse'에서 'Kubernetes / Compute Resources / Cluster', 'Kubernetes / Compute Resources / Namespace' 등 Prometheus 표준 대시보드를 선택해 클러스터/노드/파드/네임스페이스별 메트릭을 시각화할 수 있습니다.

> 참고: 관리형 Grafana는 Azure AD 계정으로 SSO(싱글사인온)되어 별도의 로그인 없이 접근할 수 있습니다.

## 6. Node Auto Provisioning(노드 자동 프로비저닝) 공식 가이드 (2025년 기준)

Node Auto Provisioning(NAP)은 AKS에서 워크로드 수요에 따라 다양한 VM 크기의 노드 풀을 자동으로 생성/확장/축소하는 기능입니다. 비용 최적화와 리소스 효율성을 위해 권장되는 Azure 기능입니다.


### Node Auto Provisioning 활성화 준비 (Azure CLI)

1. **aks-preview 확장 설치 및 최신화**

```bash

az extension add --name aks-preview || az extension update --name aks-preview

```

- aks-preview 확장이 이미 설치되어 있다면 update 명령이 실행됩니다.


2. **NodeAutoProvisioningPreview Feature Flag 등록**

```bash

az feature register --namespace "Microsoft.ContainerService" --name "NodeAutoProvisioningPreview"

az feature show --namespace "Microsoft.ContainerService" --name "NodeAutoProvisioningPreview"

# 상태가 'Registered'가 될 때까지 기다린 후, 다음 명령 실행

az provider register --namespace Microsoft.ContainerService

```

- Feature 등록 후 'Registered' 상태가 되어야 NAP 기능을 사용할 수 있습니다.


### Node Auto Provisioning 활성화 (Azure CLI)

3. **클러스터 생성 시 Node Auto Provisioning(NAP) 활성화**

   - 아래와 같이 `--node-provisioning-mode Auto` 옵션을 사용하여 클러스터를 생성하면 NAP가 활성화됩니다.
```bash
az aks create \
  --name $CLUSTER_NAME \
  --resource-group $RESOURCE_GROUP_NAME \
  --node-provisioning-mode Auto \
  --network-plugin azure \
  --network-plugin-mode overlay \
  --network-dataplane cilium \
  --generate-ssh-keys
```
- `--node-provisioning-mode Auto` 옵션이 NAP를 활성화합니다.
- 네트워크 플러그인, 데이터플레인 등은 공식 권장 옵션을 참고하세요.
- 기존 클러스터에는 현재 CLI로 NAP를 활성화할 수 없으며, 클러스터 생성 시에만 지정 가능합니다.


4. **NAP 동작 방식**
- NAP가 활성화된 클러스터는 파드가 스케줄될 수 없는 상황에서 AKS가 자동으로 다양한 VM 크기의 노드 풀을 생성하여 워크로드를 수용합니다.
- 생성된 노드 풀은 워크로드가 줄어들면 자동으로 축소(삭제)됩니다.
- NAP는 Spot 노드 풀, GPU 노드 풀 등 다양한 VM 옵션을 자동으로 활용할 수 있습니다.


### Node Auto Provisioning 테스트 방법
1. **파드 수를 대폭 늘려 노드 부족 상황 유도**
```bash
kubectl scale deployment app --replicas=30
```
- Pending 파드가 발생하면, AKS가 자동으로 새로운 노드 풀을 생성하여 파드를 스케줄링합니다.

2. **노드 풀 및 노드 상태 모니터링**
```bash
az aks nodepool list --resource-group $RESOURCE_GROUP_NAME --cluster-name $CLUSTER_NAME
kubectl get nodes -w
kubectl get pods -o wide
```
- 새로운 노드 풀이 생성되고, 노드가 추가되는 것을 확인할 수 있습니다.

3. **테스트 후 파드 수 원복**
```bash
kubectl scale deployment app --replicas=2
```
- 워크로드 감소 시 불필요한 노드 풀/노드가 자동으로 축소(삭제)됩니다.

#### 참고
- Node Auto Provisioning은 다양한 VM 크기의 노드 풀을 자동으로 생성/삭제하며, 비용 최적화와 리소스 효율성을 높일 수 있습니다.
- 자세한 내용 및 최신 옵션은 [Azure 공식 문서](https://learn.microsoft.com/en-us/azure/aks/node-autoprovision?tabs=azure-cli) 참고.

## 7. 정리
AKS 실습을 통해 클러스터 생성, 모니터링, 오토스케일링, Node Auto Provisioning 등 쿠버네티스 기반 애플리케이션의 배포와 운영 자동화 전 과정을 단계별로 실습했습니다. 각 단계에서 Azure의 관리형 서비스와 확장성, 자동화 기능을 직접 경험할 수 있습니다.생성·삭제되는 과정을 실습했습니다.

> 다음 단계에서는 CI/CD 자동화 실습을 진행합니다.